# Clasificación con datos desbalanceados

Estrategias para resolver desequilibrio de datos en Python con la librería [imbalanced-learn](https://imbalanced-learn.readthedocs.io/en/stable/).

### Descripción de los datos
El dataset contiene las transacciones de tarjeta de crédito de dos días realizadas en septiembre de 2013 por tarjetahabientes europeos. El dataset es altamente desbalanceado con un bajo porcentaje de transacciones fraudulentas entre numerosos registros de transacciones normales. La clase positiva (fraudes) representa el 0.172% (492 fraudes de 284,807 transacciones) del total de transacciones.

Los rasgos `V1`, `V2`, .... `V28` son los componentes principales (PC) obtenidos con PCA, los únicos rasgos que no han sido transformados con PCA son `Time` y `Amount`. El rasgo `Time` contiene la estampa de tiempo en segundos de cada transacción a partir de la primera transacción. El rasgo `Class` es la variable objetivo con valor 1 en caso de fraude y 0 en caso contrario.

### Requerimientos
Instalar la librería de Imbalanced Learn:<br>
`pip install -U imbalanced-learn`

## 1.0 Cargar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from statsmodels.graphics.gofplots import qqplot

In [ ]:
### RUN TO IGNORE DEPRECATION WARNINGS ###
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## 2.0 Cargar dataset
Crear un dataframe para almacenar el dataset.<br>
El dataset se encuentra disponible en: https://www.kaggle.com/mlg-ulb/creditcardfraud/data

In [ ]:
df = pd.read_csv('data\creditcard.csv') 

### 2.1 Explorar el dataframe
Visualizar 5 muestras aleatorias

In [ ]:
df.sample(5)

Identificar los nombres de todas las columnas del dataframe

In [ ]:
df.dtypes

## 3.0 Limpieza de datos
Los datasets pueden tener valores que no aparecen por una serie de razones, como observaciones que no se registraron y corrupción de datos. [Ref1](https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b), [Ref2](https://machinelearningmastery.com/handle-missing-data-python/)
### 3.1 Detectar datos faltantes en el dataframe

In [ ]:
df.isnull().any()

In [ ]:
#Contar valores núlos por rasgo
df.isnull().sum()

In [ ]:
#Para un chequeo rápido de todo el dataframe
df.isnull().values.any()

## 4.0 Desbalance de clases
Ver número de filas y muestras por clase

In [ ]:
print(df.shape)
print(pd.value_counts(df['Class']))

Visualizar desbalance mediante gráfica de barras

In [ ]:
count_classes = pd.value_counts(df['Class'])
count_classes.plot(kind = 'bar',rot=0)
LABELS = ['Normal','Fraude']
plt.xticks(range(2), LABELS)
plt.title("Frecuencia por número de observación")
plt.xlabel("Clase")
plt.ylabel("Número of observaciones");

<div class="alert alert-block alert-success">
De la gráfica anterior se puede observar que los datos están sesgados hacia la clase normal, es decir, transacciones no fradulentas.
</div>

Gráfica de barras con seaborn

In [ ]:
sns.countplot(df['Class']);

Determinar la cantidad de muestras normales y fraudulentas

In [ ]:
print('Casos normales:',len(df[df['Class'] == 0]))
print('Casos de fraude:',len(df[df['Class'] == 1]))

In [ ]:
#También se puede utilizar la función value_counts()
df['Class'].value_counts()

In [ ]:
#Proporción de transacciones normales y fradulentas
df['Class'].value_counts(normalize=True)

## 5.0 Análisis de exploratorio de datos (EDA)
### 5.1 Analizar el rasgo `Time`

In [ ]:
t=df['Time']

In [ ]:
plt.plot(t);

Convertir el tiempo de segundos a horas para facilitar la interpretación.

In [ ]:
t=t/3600

¿Cuál es la hora de la última transacción, en días?

In [ ]:
print(t.max()/24)

Crear un histograma de las transacciones durante las 48 horas:

In [ ]:
plt.figure(figsize=(12,4), dpi=60)
sns.distplot(t, bins=48, kde=True);
plt.xlim([0,48])
plt.xticks(np.arange(0,49,6))
plt.xlabel('Horas')
plt.ylabel('Número de transacciones')
plt.title('Tiempos de transacciones');

Obtener el tiempo entre transacciones (segundos):

In [ ]:
tdif=df['Time']-df['Time'].shift()

In [ ]:
plt.figure(figsize=(12,4), dpi=60)
plt.plot(tdif);

<div class="alert alert-block alert-warning">
<b>Nota</b>: Observar que hay dos lapsos con duraciones muy largas
</div> 

In [ ]:
fig=plt.figure(figsize=(12,4), dpi=60)
ax1=fig.add_subplot(111, label="1")
ax2=fig.add_subplot(111, label="2", frame_on=False)
ax3=fig.add_subplot(111, label="3", frame_on=False)
ax1.hist(t,bins=48);
ax2.plot(tdif,color='black', alpha=0.6);
ax3.plot(t,color='red');

In [ ]:
fig=plt.figure(figsize=(12,4), dpi=60)
sns.set(style="whitegrid")
sns.boxplot(tdif, linewidth=2.5);

<div class="alert alert-block alert-warning">
<b>Nota</b>: Observar outliers que muestra el diagrama de caja y bigotes
</div>

Analizar el tiempo entre transacciones de acuerdo a la clase, pero primero agregar el rasgo del tiempo entre transacciones al dataframe 

In [ ]:
df['Time_Difference'] = df['Time']-df['Time'].shift()
df.groupby('Class').Time_Difference.describe()

Mostrar gráficamente lo anterior mediante diagrama de caja y bigotes

In [ ]:
sns.boxplot(x="Class", y="Time_Difference",data=df)

<div class="alert alert-block alert-warning">
Se observa que el rango intercuartil (IQR) de ambas clases es similar; además, los valores atípicos de las diferencias de tiempo ocurren tanto en transacciones legítimas como fraudulentas. Sin embargo, algunos valores atípicos pueden indicar fraude, ya que el fraude a menudo ocurre en momentos en que hay pocas transacciones.
</div>

Ref.
1. [Two (or more) graphs in one plot with different x-axis AND y-axis scales in python](https://stackoverflow.com/questions/42734109/two-or-more-graphs-in-one-plot-with-different-x-axis-and-y-axis-scales-in-pyth)
1. [Gráficos con eje X común pero eje Y diferente: usando twinx](https://riptutorial.com/es/python/example/31794/graficos-con-eje-x-comun-pero-eje-y-diferente--usando-twinx---)

### 5.2 Analizar el rasgo `Amount`

Resumen de estadísticas

In [ ]:
df['Amount'].describe()

Se puede verificar el 75% de las transacciones están son con cantidades menores a $77.00. La siguiente figura muestra el histograma de los montos de las transacciones.

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.distplot(df['Amount'], bins=300, kde=False)
plt.ylabel('Cantidad')
plt.title('Monto de la transacción');

El histograma es difícil de leer debido a algunos valores atípicos que no podemos ver. Un diagrama de caja y bigotes mostrará los valores atípicos:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.boxplot(df['Amount'])
plt.title('Monto de la transacción');

Se puede ver que no hay valores atípicos a la izquierda y muchos valores atípicos a la derecha. Por lo tanto, las cantidades están muy sesgadas a la derecha. Para asegurar se puede calcular la asimetría:

In [ ]:
df['Amount'].skew()

Es un sesgo grande hacia la derecha. Hay que usar una transformación de potencia para aproximar los montos de las transacciones a una distribución normal. Se usará la transformación Box-Cox en SciPy, pero algunas de las cantidades son cero, así que primero hay que cambiar las cantidades para que sean positivas. Se cambiará por una cantidad muy pequeña de $10^{-9}$.

In [ ]:
#Número de ejemplos con Amount = 0
(df['Amount'] == 0).sum()


In [ ]:
#Sumar 1^-9 a la columna Amount de todas las muestras
df.loc[:,'Amount'] = df['Amount'] + 1e-9

### 5.2.1 Realizar la transformación Box-Cox al rasgo `Amount`: <br>
> 1. https://es.wikipedia.org/wiki/Transformación_Box-Cox
> 1. https://www.youtube.com/watch?v=s0XUDc_1tLM

#### Antes de la gausianización

In [ ]:
qqplot(df['Amount'], line='s')
plt.show()

In [ ]:
sns.distplot(df['Amount'], bins=48, kde=False);

In [ ]:
df['Amount'].kurtosis()

Gausianización

In [ ]:
df.loc[:,'Amount'], maxlog, (min_ci, max_ci) = sp.stats.boxcox(df['Amount'], alpha=0.01)

Después de la gausianización

In [ ]:
qqplot(df['Amount'], line='s')
plt.show()

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.distplot(df['Amount'], bins=48, kde=False);

In [ ]:
df['Amount'].kurtosis()

Mucho mejor. La distribución parece ser bimodal, lo que sugiere una división entre compras "pequeñas" y "grandes". De esta forma, la transformación de potencia eliminó la mayor parte de la asimetría de la variable `Amount`. Ahora veamos las estadísticas descriptivas de las cantidades transformadas:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.boxplot(df['Amount'])
plt.title('Monto de la transacción');

### 5.3 `Time` vs `Amount` <br>
¿Existe una relación entre los montos de la transacción y la hora del día? Crear un histograma conjunto de cajas hexagonales. Para este gráfico, se convierte cada tiempo de transacción a la hora del día en que ocurrió.

In [ ]:
sns.jointplot(t.apply(lambda x: x % 24), df['Amount'], kind='hex', stat_func=None, height=12, xlim=(0,24), ylim=(-7.5,14)).set_axis_labels('Hora del día (hr)','Monto transformado');

Los montos de las transacciones parecen estar distribuidos de manera similar a lo largo del día. Sin embargo, en las primeras horas del día, alrededor de las 5-7 AM, los montos de alrededor de 2.5 son las más comunes (recuerde que este es un valor transformado de Box-Cox). Quizás las personas están comprando su café de la mañana

### 5.4 `V1` vs `V28` 
Comparar las estadísticas descriptivas de los rasgos `V1`-`V28` transformados con PCA.

In [ ]:
pca_vars = ['V%i' % k for k in range(1,29)]

In [ ]:
#Tabla completa de estadísticas descriptivas:
df[pca_vars].describe()

Es complicado interpretar esta tabla, así que vamos a hacer algunas visualizaciones. Empezaremos por graficar las medias:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.barplot(x=pca_vars, y=df[pca_vars].mean(), color='darkblue')
plt.xlabel('Rasgo')
plt.ylabel('Media')
plt.title('Medias V1-V28');

Todos los `V1`-`V28` tienen aproximadamente una media cero. Ahora graficar las desviaciones estándar:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.barplot(x=pca_vars, y=df[pca_vars].std(), color='darkred')
plt.xlabel('Rasgo')
plt.ylabel('Deviación estándar')
plt.title('Desviación estándar V1-V28');

Las componentes principales tienen aproximadamente una varianza unitaria, que oscila entre ~0.3 y ~1.9. A continuación, se grafican las asimetrías:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.barplot(x=pca_vars, y=df[pca_vars].skew(), color='darkgreen')
plt.xlabel('Rasgo')
plt.ylabel('Asimetría')
plt.title('Asimetría (skewnesses) V1-V28 ');

Algunas de las PC están significativamente sesgadas. Graficar un histograma de una de las variables sesgadas, por ej. `V8`, para ver la distribución en detalle.

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.distplot(df['V8'], bins=300, kde=False)
plt.ylabel('Magnitud')
plt.title('V8')

El histograma no muestra valores atípicos. Probar con un diagram de caja y bigotes:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.boxplot(df['V8'])
plt.title('V8');

<div class="alert alert-block alert-info">
El diagrama de caja también es difícil de interpretar debido al gran número de valores atípicos, lo que indica una alta curtosis en V8. Esto sugiere graficar las curtosis de las PC. El método de kurtosis empleado en Pandas es la definición de Fisher, donde la distribución normal estándar tiene kurtosis 0.
</div>

Observar la escala logarítmica en el eje $y$ en la gráfica siguiente:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
plt.yscale('log')
sns.barplot(x=pca_vars, y=df[pca_vars].kurtosis(), color='darkorange')
plt.xlabel('Rasgo')
plt.ylabel('Asimetría')
plt.title('Asimetría (skewnesses) V1-V28 ');

Se ha descubierto que muchas de las PC son de cola pesada (heavy-tailed). El gran número de valores atípicos en `V1`-`V28` hace necesario considerar estadísticas descriptivas robustas. Graficar las medianas:

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.barplot(x=pca_vars, y=df[pca_vars].median(), color='darkblue')
plt.xlabel('Rasgo')
plt.ylabel('Mediana')
plt.title('Mediana de V1-V28')

Las medianas también son muy cercanas a cero. A continuación, analicemos los rangos intercuartiles (IQR)*:
* Pandas no tiene un método IQR incorporado, pero se puede usar el método de cuantiles para calcular el IQR.

In [ ]:
plt.figure(figsize=(12,4), dpi=80)
sns.barplot(x=pca_vars, y=df[pca_vars].quantile(0.75) - df[pca_vars].quantile(0.25), color='darkred')
plt.xlabel('Rasgo')
plt.ylabel('IQR')
plt.title('IQRs V1-V28');

### 5.5 Matriz de correlaciones

In [ ]:
#Primero eliminar rasgo adicional
df=df.drop(['Time_Difference'],axis=1)

In [ ]:
# Correlation matrix
corrmat = df.corr()
fig = plt.figure(figsize = (12, 9))
sns.heatmap(corrmat, cmap='seismic')
plt.show()

Encontrar los rasgos gausianizados que presentan la mayor correlación

## 6. Modelado
Ahora estamos listos para crear modelos de aprendizaje automático para predecir si una transacción es fraudulenta.<br> Entrenaremos a los siguientes modelos:
1. Regresión logística
1. Clasificador de soporte vectorial

### 6.1 Clasificación con datos desbalanceados
Utilizar la regresión logística para hacer la clasificación de estos datos desbalanceados.

#### 6.1.1 Preparación de datos
Antes de ejecutar el algoritmo, normalizar el rasgo `Amount` y eliminar el rasgo `Time`.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
df=df.drop(['Time','Amount'],axis=1)
X=df.drop(['Class'],axis=1)
y=df['Class']

> #### Notas sobre el uso de StandardScaler
__`StandardScaler`__ : Transforma los datos de tal manera que su media sea 0 y su desviación estándar 1. En resumen, estandariza los datos. La estandarización es útil para datos que tienen valores negativos. Ajusta los datos a una distribución normal. Es más útil en la clasificación que en la regresión. 
StandardScaler estandariza un rasgo restando la media y luego escalándolo a una varianza unitaria. La desviación unitaria significa dividir todos los valores por la desviación estándar.<br><br>
__`Normalizer`__ : Comprime los datos entre 0 y 1, esto es, efectua una normalización. Debido a la disminución del rango y la magnitud, los gradientes en el proceso de entrenamiento no se desbordarán y no se obtendrán valores más altos de pérdida. Es más útil en la regresión que en la clasificación.

#### 6.1.2 Particionar dataset (entrenamiento y prueba)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

#### 6.1.3 Aplicar el algoritmo de regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score,accuracy_score,precision_score,f1_score,confusion_matrix
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

#### 6.1.4 Evaluación del algoritmo

In [ ]:
conf = confusion_matrix(y_test,y_pred)
print('Matriz de confusión:')
print(conf)
print('Exactitud =',accuracy_score(y_test,y_pred))
print('Sensibilidad [clase 0 , clase 1] =',recall_score(y_test,y_pred,average=None))
print ('Precision =', precision_score(y_test, y_pred, average=None))
print ('F1 =', f1_score(y_test, y_pred, average=None))

#### 6.1.5 Observaciones:
1. __Exactitud__: Podría creerse que el algoritmo está funcionando extremadamente bien. Pero no es verdad. Debido a que la mayoría de las etiquetas son 0, incluso con predicciones aleatorias dan una exactitud del 99%. Por lo tanto, se necesita una mejor métrica para entender el rendimiento del modelo.
1. __Sensibilidad__: Como se puede observar en los resultados, la sensibilidad de 1 es sólo 0.61904762 comparado con el 99% de 0. Así que el modelo no está haciendo un buen trabajo en el reconocimiento de fraudes. Esto muestra cómo los datos desbalanceados están afectando la precisión del modelo.

In [ ]:
#La función classification_report nos proporciona las métricas anteriores
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
conf = pd.DataFrame(conf, range(2),range(2))
fig = plt.figure(figsize = (3, 3))
sns.heatmap(conf, annot=True, fmt='g', annot_kws={"size": 14}, cmap='seismic', cbar=False)
plt.title('Matriz de confusión');

#### 6.1.6 validación cruzada de K iteraciones (K-fold cross-validation)

Para evaluar el rendimiento del modelo de regresión logística para todo el conjunto de datos y no sólo para el conjunto de test (p. ej., el 30%).

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=7)
metrica = 'precision'
results = model_selection.cross_val_score(logreg, X_train, y_train, cv=kfold, scoring=metrica)
print('Valores de la métrica en cada iteración: ',results)
print("Validación cruzada de 10 iteraciones, promedio de la métrica: %.3f" % (results.mean()))
print("Validación cruzada de 10 iteraciones, desviación estándard de la métrica: %.3f" % (results.std()))

#### 6.1.7 Curvas

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
# Calcular TFP, TVP, thresholds y roc auc
y_pred_prob = logreg.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.plot([0,1],[0,1],'k--', label='Clasificador aleatorio')
roc_auc = roc_auc_score(y_test, y_pred)
plt.plot(fpr, tpr,'r-', label='Regresión logística')
plt.plot([0,0,1],[0,1,1],'g-',label='Clasificador ideal')
plt.title('Curva ROC')
plt.xlabel('Tasa de falsos positivos')
plt.ylabel('Tasa de verdaderos positivos')
plt.legend(loc="lower right")
plt.show()
print('AUC =',roc_auc)

In [ ]:
logreg.predict_proba(X_test).shape

In [ ]:
from sklearn.metrics import precision_recall_curve,average_precision_score
lr_precision, lr_recall,_ = precision_recall_curve(y_test, y_pred_prob)
plt.plot(lr_recall, lr_precision, label='Regresión logística')
plt.plot([1,1,0],[0,1,1],'g-',label='Clasificador ideal')
plt.title('Curva PR')
plt.xlabel('Recall')
plt.ylabel('Precision');
plt.legend(loc="lower left")
plt.show()
print('AUC =',average_precision_score(y_test, y_pred_prob))

### 6.2 Clasificación con datos balanceados

#### 6.2.1 Sobremuestreo de datos
Se hará un sobremuestreo a la clase minoritaria con SMOTE (**_Synthetic Minority Over-sampling Technique_**), funciona buscando dos vecinos cercanos en una clase minoritaria, produciendo un nuevo punto intermedio entre los dos puntos existentes y añadiendo ese nuevo punto a la muestra (rasgos continuos).

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=4001,kind='regular')
X_train_sob, y_train_sob = sm.fit_sample(X_train, y_train)
pd.value_counts(y_train_sob)

<div class="alert alert-block alert-success">
Nota: el algoritmo <b>SMOTE</b> ha sobremuestreado las instancias de las minorías y las ha hecho iguales a las de la clase mayoritaria. Ambas categorías tienen la misma cantidad de registros. Concretamente, la clase minoritaria se ha incrementado hasta el número total de clases mayoritarias.
Ahora calcular la precisión y sensibilidad después de aplicar el algoritmo SMOTE (sobremuestreo).
</div>

#### 6.2 Predicción y evaluación - Conjunto remuestreado
A continuación se usará la regresión logística para probar el rendimiento en el mismo conjunto de validación.

In [ ]:
#logreg_sob = LogisticRegression(solver='lbfgs',random_state=5001)
from sklearn import linear_model
logreg_sob = linear_model.LogisticRegression()
logreg_sob.fit(X_train_sob,y_train_sob)
y_pred_sob = logreg_sob.predict(X_test)

In [ ]:
# Imprimir el reporte de clasificación 
#print(classification_report(y_test, y_pred_sob)) 
recall_score(y_test, y_pred_sob, average=None)

Se ha reducido la exactitud al 98% en comparación con el modelo anterior, pero el valor de sensibilidad de la clase minoritaria ha mejorado al 92%. Este es un buen modelo comparado con el anterior. La sensibilidad es excelente.

In [ ]:
y_pred_prob_sob = logreg_sob.predict_proba(X_test)[:,1]
fpr_res, tpr_res, thresholds_res = roc_curve(y_test, y_pred_prob_sob)
plt.plot(fpr_res, tpr_res,'r-', label='Regresión logística')
roc_auc = roc_auc_score(y_test, y_pred_prob_sob)
plt.show()
print('AUC =',roc_auc)

In [ ]:
lr_precision_sob, lr_recall_sob,_ = precision_recall_curve(y_test, y_pred_prob_sob)
plt.plot(lr_recall_sob, lr_precision_sob, label='Logistic')
plt.show()
print('AUC =',average_precision_score(y_test, y_pred_prob_sob))

#### 6.2.2 Submuestreo de datos
El algoritmo `NearMiss` es una técnica de submuestreo que se basa en el clasificador KNN.

In [ ]:
from imblearn.under_sampling import NearMiss 
nr = NearMiss() 
X_train_sub, y_train_sub = nr.fit_sample(X_train, y_train) 
pd.value_counts(y_train_sub)

Entrenar el modelo

In [ ]:
# Entrenar el modelo
logreg2 = LogisticRegression(solver='lbfgs',random_state=1)
logreg2.fit(X_train_sub, y_train_sub) 
y_pred_sub = logreg2.predict(X_test)

Imprimir el reporte de clasificación

In [ ]:
print(classification_report(y_test, y_pred_sub)) 

#### 6.2.3 Usando `Class Weight`

In [ ]:
lr_balanced = LogisticRegression(solver='lbfgs',class_weight = 'balanced')
lr_balanced.fit(X_train,y_train)
y_balanced_pred = lr_balanced.predict(X_test)
print(recall_score(y_test,y_balanced_pred))
print(accuracy_score(y_test,y_balanced_pred))

## 7 Selección del modelo
### 7.1 Grid Search
Mediante esta técnica se puede encontrar la mejor combinación de hiperparámetros de un algoritmo con el propósito de reducir el sobreajuste.
Se basa en una búsqueda exhaustiva por el paradigma de fuerza bruta en el que se especifica una lista de valores para diferentes hiperparámetros y el método evalúa el rendimiento del modelo para cada combinación de éstos parámetros para obtener el conjunto óptimo que brinda el mayor rendimiento.

In [ ]:
#importar la clase GridSearchCV de la librería sklearn.model_selection
from sklearn.model_selection import GridSearchCV

Primeramente, crear un diccionario de los parámetros y sus valores que se desea probar para obtener el mejor rendimiento. 
Los detalles de todos los parámetros para el algoritmo de bosque aleatorio están disponibles en los documentos de [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [ ]:
grid_param={'C':np.logspace(-3,3,7), 'penalty':["l1","l2"]}# l1 lasso l2 ridge
#grid_param={'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
#grid_param={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

Definir la búsqueda de rejilla

In [ ]:
logreg_cv=GridSearchCV(logreg_sob,grid_param,cv=5, scoring='recall',verbose=1)

Hacer el ajuste

In [ ]:
best_model = logreg_cv.fit(X_train_sob,y_train_sob)

Imprimir los valores de los hiperparámetros del mejor modelo

In [ ]:
print('Mejor penalización:', best_model.best_estimator_.get_params()['penalty'])
print('Mejor C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
print("Mejores hiperparámetros: ",logreg_cv.best_params_)
print("Sensibilidad :",logreg_cv.best_score_)

#### 6.3.2 Utilizar los hiperparámetros obtenidos

In [ ]:
logreg_sob = linear_model.LogisticRegression(C=100,penalty='l2')
logreg_sob.fit(X_train_sob,y_train_sob)
y_pred_sob = logreg_sob.predict(X_test)

In [ ]:
# Imprimir el reporte de clasificación 
print(classification_report(y_test, y_pred_sob)) 

In [ ]:
print ('Sensibilidad [0/1] = ',recall_score(y_test, y_pred_sob, average=None))

In [ ]:
%reset -f